# Desafio - Estágio
Este projeto objetiva tratar, explorar e analizar os dados referentes a viagens realizadas a serviço por servidores públicos. Serão analisados dois triênios (2017-2019) e (2020-2022).
Os datasets foram extraídos do seguinte site: https://portaldatransparencia.gov.br/download-de-dados/viagens  

## Bibliotecas necessárias

In [174]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Importação dos datasets
No site do Portal da Transparência os arquivos estão organizados por exercício (ano). Para cada exercício há quatro planilhas (Pagamento, Passagem, Trecho e Viagem). Como o objetivo deste projeto é fazer a análise de dois triênios, foi feito o download de seis diretórios. Totalizando, assim, 24 arquivos. Abaixo o script de importação e união dos datasets. Por ano é executada a união de quatro datasets em apenas um, totalizando seis planilhas a serem trabalhadas.

In [3]:
pagamento = pd.read_csv('datasets/2022/pagamento_2022.csv')
passagem = pd.read_csv('datasets/2022/passagem_2022.csv')
trecho = pd.read_csv('datasets/2022/trecho_2022.csv')
viagem = pd.read_csv('datasets/2022/viagem_2022.csv')



In [4]:
#é usada a função merge() para fazer a junção do tipo inner join com base na coluna "Id", que é comum a todas as tabelas

tabela_final = pd.merge(pagamento, passagem, on='Id')
tabela_final = pd.merge(tabela_final, trecho, on='Id')
tabela_final = pd.merge(tabela_final, viagem, on='Id')

#abaixo é apagado registros com valores de Id duplicados
tabela_final = tabela_final.drop_duplicates(subset='Id')


In [22]:
#o comando abaixo amplia a quantidade máxima de colunas visualizadas
pd.set_option('display.max_columns', 50)

In [56]:
#exportar dataframe para arquivo .csv
#tabela_final.to_csv('datasets/2022/tabela_final22.csv', index=False, encoding='utf-8')

## Dicionário de Dados

| Coluna | Descrição ||
|--- |--- |--- |
| Id | Número que identifica o processo de concessão da viagem  |
| Cod_org_superior | Código do Órgão Superior que custeou a despesa |
| Nome_org_superior | Nome do Órgão Superior |
| Cod_org_pagador | Código do Órgão que pagou a despesa  |
| Nome_org_pagador | Nome do órgão |
| Cod_ug_pagadora | Código da Unidade Gestora que pagou a despesa |
| Nome_ug_pagadora | Nome da Unidade Gestora |
| Tipo_pagamento | Tipo da despesa paga pelo órgão (diária, passagem, seguro, etc.) |
| Valor | Valor da despesa paga |
| Meio_transporte | Meio de transporte da passagem |
| Pais_origem_ida | País de origem ida |
| UF_origem_ida | UF de origem na ida |
| Cidade_origem_ida | Cidade de origem na ida |
| Pais_destino_ida | País de destino na ida |
| UF_destino_ida | UF de destino na ida |
| Cidade_destino_ida | Cidade de destino na ida  |
| Pais_origem_volta | País de origem na volta |
| UF_origem_volta | UF de origem na volta |
| Cidade_origem_volta | Cidade de origem na volta |
| Pais_destino_volta | País de destino na volta |
| UF_destino_volta | UF de destino na volta |
| Cidade_destino_volta | Cidade de destino na volta |
| Valor_passagem | Valor da passagem |
| Taxa_servico | Taxa de serviço da passagem |
| Trecho | Sequência do trecho na viagem |
| Data_origem | Data de saída da origem |
| Pais_origem | País de origem  |
| UF_origem | UF de origem  |
| Cidade_origem | Cidade de origem  |
| Data_destino | Data de chegada no destino  |
| Pais_destino | País de destino  |
| UF_destino | UF de destino   |
| Cidade_destino | Cidade de destino   |
| Diárias | Número de diárias do trecho  |
| Missão | Houve missão no trecho? ‘Sim’ ou ‘Não’  |
| Situação | Situação da viagem: ´não realizada´; ´realizada’  |
| Cod_org_solicitante | Código do Órgão que solicitou a viagem  |
| Nome_org_solicitante | Nome do Órgão que solicitou a viagem  |
| CPF | CPF da pessoa que realizou a viagem   |
| Nome | Nome do viajante   |
| Cargo | Cargo do viajante   |
| Periodo_inicio | Data de início de afastamento do servidor   |
| Periodo_fim | Data de fim de afastamento do servidor   |
| Valor_diárias | Valor de diárias pagas pelo órgão, se houver   |
| Valor_passagens | Valor de passagens pagas pelo órgão, se houver   |
| Valor_devolução | Valor devolvido, se houver   |
| Valor_outros_gastos | Valor de outros gastos pagos pelo órgão, se houver   |

      


Abaixo é feita a importação dos datasets que englobam os dados a serem tratados

In [32]:
df17 = pd.read_csv('datasets/tabela_final17.csv')
df18 = pd.read_csv('datasets/tabela_final18.csv')
df19 = pd.read_csv('datasets/tabela_final19.csv')
df20 = pd.read_csv('datasets/tabela_final20.csv')
df21 = pd.read_csv('datasets/tabela_final21.csv')
df22 = pd.read_csv('datasets/tabela_final22.csv')

## Tratando os Dados


Foi realizada a eliminação de forma manual de alguns atributos dos datasets. Isso se deu por motivos de que tais colunas estavam quase que completamente com valores de "Sem informação" e/ou não eram colunas relevantes para a análise presente. Colunas eliminadas: **Pais_origem_volta, UF_origem_volta, Cidade_origem_volta, Pais_destino_volta, UF_destino_volta, Cidade_destino_volta, Cod_org_solicitante, Nome_org_solicitante** e **CPF.**

### Classe das Variáveis 
Alguns atributos estão classificados como do tipo object, quando na verdade devem ser do tipo float. São eles: **Valor, Valor_passagem, Taxa_servico, Valor_diárias, Valor_passagens, Valor_devolução** e **Valor_outros_gastos.** 
Alguns atributos estão classicados como do tipo object, quando devem ser do tipo date. Saõ eles: **Data_origem, Data_destino, Periodo_inicio** e **Periodo_fim.** 

Como os seis datasets são estruturalmente iguais, foram elaboradas funções para tratar alguns aspectos dos dados.

Função para substituir a vírgula pelo ponto e converter uma lista de colunas para float.

In [41]:
#função para remover espaços das colunas tipo object
def remover_espacos(dfs):
    for df in dfs:
        df.columns = df.columns.str.strip()
        df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)



In [42]:
#lista dos dataframes
dfs = [df17, df18, df19, df20, df21, df22]
#chamando a função
remover_espacos(dfs)

In [43]:
def mudar_classe(df, colunas):
    for coluna in colunas:
        df[coluna] = df[coluna].str.replace(',', '.').astype(float)
    return df

In [86]:
colunas = ['Valor', 'Valor_passagem', 'Taxa_servico', 'Valor_diárias', 'Valor_passagens', 'Valor_devolução', 'Valor_outros_gastos']


In [45]:
df17 = mudar_classe(df17, colunas)

In [46]:
df18 = mudar_classe(df18, colunas)

In [ ]:
df19 = mudar_classe(df19, colunas)

In [47]:
df20 = mudar_classe(df20, colunas)

In [48]:
df21 = mudar_classe(df21, colunas)

In [49]:
df22 = mudar_classe(df22, colunas)

Função para converter uma lista de colunas do tipo object para o tipo date.

In [50]:
def converter_date(df, colunas):
    for coluna in colunas:
        df[coluna] = pd.to_datetime(df[coluna], format='%d/%m/%Y')
    return df

In [51]:
#lista de colunas
colunas = ['Data_origem', 'Data_destino', 'Periodo_inicio', 'Periodo_fim']


In [52]:
df17 = converter_date(df17, colunas)

In [53]:
df18 = converter_date(df18, colunas)

In [54]:
df19 = converter_date(df19, colunas)

In [55]:
df20 = converter_date(df20, colunas)

In [56]:
df21 = converter_date(df21, colunas)

In [57]:
df22 = converter_date(df22, colunas)

Função para substituir na coluna **Tipo_pagamento** os valores "Serviço correlato: seguro" para SEGURO.

In [58]:
def seguro (dfs, coluna):
    for df in dfs:
        df[coluna] = df[coluna].replace('Serviço correlato: seguro','SEGURO')

In [59]:
#chamando a função
seguro(dfs, 'Tipo_pagamento')

Função para renomear a coluna **Situação** para **Realizada** 

In [60]:
def renomear_coluna(dfs, coluna, novo_nome):
    for df in dfs:
        df.rename(columns={coluna: novo_nome}, inplace=True)

In [61]:
renomear_coluna(dfs, 'Situação', 'Realizada')

Função para substituir os valores da coluna **Realizada** de Realizada para Sim, e Não realizada para Não.

In [62]:
def substituir(dfs, coluna, dicionario):
    for df in dfs:
        df[coluna] = df[coluna].replace(dicionario)

In [63]:
dicionario = {'Realizada':'Sim','Não realizada':'Não'}

In [64]:
#chamando a função
substituir(dfs, 'Realizada', dicionario)

### Valores Ausentes

Foi rodado o comando abaixo para descobrir quais colunas contam com valores nulos e em que quantidade. Para os 6 datasets, há sempre e somente 5 colunas que contém valores nulos. São elas: **UF_origem_ida, UF_destino_ida, UF_origem, UF_destino** e **Cargo**. A coluna **Cargo** conta com mais de 50% de seus valores classificados como nulos. Por este motivo, tal atributo não será utilizado na análise.
As outras 4 colunas contam com aproximadamente 5% de seus valores classificados como nulos. Quando estes atributos estão em branco é porque se trata de uma viagem com origem ou destino em país estrangeiro.

In [ ]:
df17.isnull().sum()
df18.isnull().sum()
df19.isnull().sum()
df20.isnull().sum()
df21.isnull().sum()
df22.isnull().sum()


Função para eliminar a coluna **Cargo**

In [65]:
def eliminar_cargo(dfs, coluna):
    for df in dfs:
        df.drop(coluna, axis=1, inplace=True)

In [66]:
#chamando a função
eliminar_cargo(dfs, 'Cargo')

Função para inserir nas colunas **UF_origem_ida, UF_destino_ida, UF_origem, UF_destino** o valor de "Internacional" onde os campos estiverem vazios.

In [67]:
def inserir_valor(dfs, colunas, valor):
    for df in dfs:
        for coluna in colunas:
            df[coluna].fillna(valor, inplace=True)

In [68]:
#lista de colunas
colunas = ['UF_origem_ida','UF_destino_ida','UF_origem','UF_destino']
#chamando a função
inserir_valor(dfs, colunas, 'Internacional')


Exportação dos dataframes limpos para arquivos .csv

In [94]:
df17.to_csv('datasets/tabela_clean17.csv', index=False, encoding='utf-8')
df18.to_csv('datasets/tabela_clean18.csv', index=False, encoding='utf-8')
df19.to_csv('datasets/tabela_clean19.csv', index=False, encoding='utf-8')
df20.to_csv('datasets/tabela_clean20.csv', index=False, encoding='utf-8')
df21.to_csv('datasets/tabela_clean21.csv', index=False, encoding='utf-8')
df22.to_csv('datasets/tabela_clean22.csv', index=False, encoding='utf-8')

## Análise

Importação dos datasets tratados

In [155]:
df17_clean = pd.read_csv('datasets/tabela_clean17.csv')
df18_clean = pd.read_csv('datasets/tabela_clean18.csv')
df19_clean = pd.read_csv('datasets/tabela_clean19.csv')
df20_clean = pd.read_csv('datasets/tabela_clean20.csv')
df21_clean = pd.read_csv('datasets/tabela_clean21.csv')
df22_clean = pd.read_csv('datasets/tabela_clean22.csv')

Abaixo é determinado o nome do servidor que mais gastou com viagens no triênio (2017-2019)

In [186]:
df1 = pd.DataFrame(df17_clean, columns=['Nome', 'Taxa_servico','Valor_diárias', 'Valor_passagens', 'Valor_outros_gastos'])
df2 = pd.DataFrame(df18_clean, columns=['Nome', 'Taxa_servico','Valor_diárias', 'Valor_passagens', 'Valor_outros_gastos'])
df3 = pd.DataFrame(df19_clean, columns=['Nome', 'Taxa_servico','Valor_diárias', 'Valor_passagens', 'Valor_outros_gastos'])

trienio1 = pd.concat([df1, df2, df3])

In [117]:
#calcula o gasto total
trienio1['Gasto_total'] = trienio1['Taxa_servico'] + trienio1['Valor_diárias'] + trienio1['Valor_passagens'] + trienio1['Valor_outros_gastos']


# Agrupa o dataframe pelo nome da pessoa e calcule a soma dos gastos totais para cada pessoa
gastos_por_pessoa = trienio1.groupby('Nome')['Gasto_total'].sum().reset_index()

# Ordena o dataframe em ordem decrescente com base nos valores de gasto total
gastos_por_pessoa = gastos_por_pessoa.sort_values('Gasto_total', ascending=False)

pessoa_maior_gasto = gastos_por_pessoa.iloc[0]['Nome']
valor_maior_gasto = gastos_por_pessoa.iloc[0]['Gasto_total']



In [120]:
print(pessoa_maior_gasto, valor_maior_gasto)

DECIO FABRICIO ODDONE DA COSTA 457018.56


Conferindo os registros do servidor

In [187]:
nome  = 'DECIO FABRICIO ODDONE DA COSTA'
flt = trienio1[trienio1['Nome'] == nome]
flt

,Nome,Taxa_servico,Valor_diárias,Valor_passagens,Valor_outros_gastos
2340,DECIO FABRICIO ODDONE DA COSTA,0.0,537.91,770.39,0.00
13730,DECIO FABRICIO ODDONE DA COSTA,0.0,568.41,1291.25,0.00
16016,DECIO FABRICIO ODDONE DA COSTA,0.0,653.73,1381.97,0.00
21197,DECIO FABRICIO ODDONE DA COSTA,0.0,6478.62,7120.12,71.74
33810,DECIO FABRICIO ODDONE DA COSTA,0.0,537.91,667.99,0.00
...,...,...,...,...,...
108947,DECIO FABRICIO ODDONE DA COSTA,85.0,0.00,722.00,0.00
113441,DECIO FABRICIO ODDONE DA COSTA,49.0,0.00,2472.00,0.00
120691,DECIO FABRICIO ODDONE DA COSTA,7.0,0.00,15660.00,0.00
120692,DECIO FABRICIO ODDONE DA COSTA,3.0,0.00,2826.00,0.00


Abaixo é determinado o nome do servidor que mais gastou com viagens no triênio (2020-2022)

In [128]:
df4 = pd.DataFrame(df20_clean, columns=['Nome', 'Taxa_servico','Valor_diárias', 'Valor_passagens', 'Valor_outros_gastos'])
df5 = pd.DataFrame(df21_clean, columns=['Nome', 'Taxa_servico','Valor_diárias', 'Valor_passagens', 'Valor_outros_gastos'])
df6 = pd.DataFrame(df22_clean, columns=['Nome', 'Taxa_servico','Valor_diárias', 'Valor_passagens', 'Valor_outros_gastos'])

trienio2 = pd.concat([df4, df5, df6])

In [129]:
#calcula o gasto total
trienio2['Gasto_total'] = trienio2['Taxa_servico'] + trienio2['Valor_diárias'] + trienio2['Valor_passagens'] + trienio2['Valor_outros_gastos']


# Agrupa o dataframe pelo nome da pessoa e calcule a soma dos gastos totais para cada pessoa
gastos_por_pessoa = trienio2.groupby('Nome')['Gasto_total'].sum().reset_index()

# Ordena o dataframe em ordem decrescente com base nos valores de gasto total
gastos_por_pessoa = gastos_por_pessoa.sort_values('Gasto_total', ascending=False)

pessoa_maior_gasto = gastos_por_pessoa.iloc[0]['Nome']
valor_maior_gasto = gastos_por_pessoa.iloc[0]['Gasto_total']

In [130]:
print(pessoa_maior_gasto, valor_maior_gasto)

BENTO COSTA LIMA LEITE DE ALBUQUERQUE JUNIOR 1028337.8


Conferindo os registros do servidor

In [134]:
nome  = 'BENTO COSTA LIMA LEITE DE ALBUQUERQUE JUNIOR'
flt = trienio2[trienio2['Nome'] == nome]

flt

,Nome,Taxa_servico,Valor_diárias,Valor_passagens,Valor_outros_gastos,Gasto_total
4275,BENTO COSTA LIMA LEITE DE ALBUQUERQUE JUNIOR,0.0,1685.49,3339.42,0.00,5024.91
4301,BENTO COSTA LIMA LEITE DE ALBUQUERQUE JUNIOR,0.0,7605.68,24051.97,393.82,32051.47
5255,BENTO COSTA LIMA LEITE DE ALBUQUERQUE JUNIOR,0.0,75.92,1506.57,0.00,1582.49
9778,BENTO COSTA LIMA LEITE DE ALBUQUERQUE JUNIOR,0.0,1680.54,3132.42,0.00,4812.96
11321,BENTO COSTA LIMA LEITE DE ALBUQUERQUE JUNIOR,0.0,539.36,2244.42,0.00,2783.78
...,...,...,...,...,...,...
41785,BENTO COSTA LIMA LEITE DE ALBUQUERQUE JUNIOR,0.0,786.29,3451.98,0.00,4238.27
42436,BENTO COSTA LIMA LEITE DE ALBUQUERQUE JUNIOR,0.0,269.68,5067.86,0.00,5337.54
42663,BENTO COSTA LIMA LEITE DE ALBUQUERQUE JUNIOR,0.0,0.00,63997.61,484.44,64482.05
44732,BENTO COSTA LIMA LEITE DE ALBUQUERQUE JUNIOR,0.0,0.00,3290.66,0.00,3290.66


Qual a Unidade Gestora (UG) que mais gastou com viagens no triênio (2017-2019)?

In [142]:
df1 = pd.DataFrame(df17_clean, columns=['Nome_ug_pagadora', 'Taxa_servico','Valor_diárias', 'Valor_passagens', 'Valor_outros_gastos'])
df2 = pd.DataFrame(df18_clean, columns=['Nome_ug_pagadora', 'Taxa_servico','Valor_diárias', 'Valor_passagens', 'Valor_outros_gastos'])
df3 = pd.DataFrame(df19_clean, columns=['Nome_ug_pagadora', 'Taxa_servico','Valor_diárias', 'Valor_passagens', 'Valor_outros_gastos'])

trienio1 = pd.concat([df1, df2, df3])

In [144]:
trienio1['Gasto_total'] = trienio1['Taxa_servico'] + trienio1['Valor_diárias'] + trienio1['Valor_passagens'] + trienio1['Valor_outros_gastos']
# Agrupa o dataframe pelo nome da UG e calcule a soma dos gastos totais para cada UG
gastos_ug = trienio1.groupby('Nome_ug_pagadora')['Gasto_total'].sum().reset_index()

# Ordena o dataframe em ordem decrescente com base nos valores de gasto total
gastos_ug = gastos_ug.sort_values('Gasto_total', ascending=False)

ug_maior_gasto = gastos_ug.iloc[0]['Nome_ug_pagadora']
valor_maior_gasto = gastos_ug.iloc[0]['Gasto_total']

In [146]:
print(ug_maior_gasto, valor_maior_gasto)

INST.NACIONAL DE EST.E PESQUISAS EDUCACIONAIS 62438848.64


Qual a Unidade Gestora (UG) que mais gastou com viagens no triênio (2020-2022)?

In [149]:
df4 = pd.DataFrame(df20_clean, columns=['Nome_ug_pagadora', 'Taxa_servico','Valor_diárias', 'Valor_passagens', 'Valor_outros_gastos'])
df5 = pd.DataFrame(df21_clean, columns=['Nome_ug_pagadora', 'Taxa_servico','Valor_diárias', 'Valor_passagens', 'Valor_outros_gastos'])
df6 = pd.DataFrame(df22_clean, columns=['Nome_ug_pagadora', 'Taxa_servico','Valor_diárias', 'Valor_passagens', 'Valor_outros_gastos'])

trienio2 = pd.concat([df4, df5, df6])

In [150]:
trienio2['Gasto_total'] = trienio2['Taxa_servico'] + trienio2['Valor_diárias'] + trienio2['Valor_passagens'] + trienio2['Valor_outros_gastos']

gastos_ug = trienio2.groupby('Nome_ug_pagadora')['Gasto_total'].sum().reset_index()


gastos_ug = gastos_ug.sort_values('Gasto_total', ascending=False)

ug_maior_gasto = gastos_ug.iloc[0]['Nome_ug_pagadora']
valor_maior_gasto = gastos_ug.iloc[0]['Gasto_total']

In [151]:
print(ug_maior_gasto, valor_maior_gasto)

FUNDO NACIONAL DE SEGURANCA PUBLICA - FNSP 133011826.96


Os resultados são coerentes, dado que o **INST.NACIONAL DE EST.E PESQUISAS EDUCACIONAIS** pertence ao MEC, e o **FUNDO NACIONAL DE SEGURANCA PUBLICA - FNSP** pertence ao Ministério da Justiça. Dois dos ministérios com mais verbas e mais pessoal.

Qual é o tempo em média que o servidor permanece afastado de suas funções?

In [161]:
df17_clean['Diferenca'] = df17_clean['Periodo_fim'] - df17_clean['Periodo_inicio']
df18_clean['Diferenca'] = df18_clean['Periodo_fim'] - df18_clean['Periodo_inicio']
df19_clean['Diferenca'] = df19_clean['Periodo_fim'] - df19_clean['Periodo_inicio']
df20_clean['Diferenca'] = df20_clean['Periodo_fim'] - df20_clean['Periodo_inicio']
df21_clean['Diferenca'] = df21_clean['Periodo_fim'] - df21_clean['Periodo_inicio']
df22_clean['Diferenca'] = df22_clean['Periodo_fim'] - df22_clean['Periodo_inicio']

In [163]:
dfs = [df17_clean, df18_clean, df19_clean, df20_clean, df21_clean, df22_clean]

medias = []
for df in dfs:
    media = df['Diferenca'].mean()
    medias.append(media)

In [167]:
#converte em uma series do pandas
medias_series = pd.Series(medias)

# extrair apenas a parte dos dias das médias
medias_dias = medias_series.dt.days

In [168]:
medias_dias

0    5
1    6
2    5
3    8
4    8
5    8
dtype: int64

Entre 2017-2019 a média de tempo em afastamento era entre 5 e 6 dias. Já durante o triênio da pandemia de Covid-19 esta mesma média subiu e se estabilizou em 8 dias. Muito possivelmente este fato é explicado pela adoção de regimes de lockdown entre os diversos governos do país e do mundo. O que acarretou que muitos viajantes acabaram ficando mais dias nos destinos.

# Conclusões

Alguns insights relevantes:
* Como já era esperado houve uma forte queda no número de viagens;
* Atualmente o servidor fica em média 8 dias afastado para viagem;
* Os países mais visitados são: EUA, França, Portugal, China e Argentina;
* As cidades brasileiras mais visitadas são: Brasília, São Paulo, Manaus, Rio de Janeiro, BH e Porto Alegre;
* As UFs onde mais há partidas são: DF, RJ, RS e AM;
* Mais de 90% das viagens ocorreram por via aérea;
* Cerca de 15% das viagens ocorreram sem nenhum tipo de missão oficial;
* No ano de 2020 (ano mais intenso da pandemia) apenas 20% das viagens não ocorreram;
* Entre 2017-2019 o servidor DECIO FABRICIO ODDONE DA COSTA foi o que mais gastou em viagens;
* entre 2020-2022 o servidor BENTO COSTA LIMA LEITE DE ALBUQUERQUE JUNIOR foi o que mais gastou em viagens.


# Observações

Eu desenvolvi um dashboard em streamlit, porém não consegui realizar o deploy. Tentei, então, criar um arquivo python executável. No entanto, também não obtive sucesso.

Por isso, abaixo eu anexei os gráficos contidos no dashboard.

O código-fonte do dashboard "viagens.py" está no repositório.

![sidebar](sidebar.PNG)

![sup](sup.PNG)

![inf](inf.PNG)